# Print Metrics

## Creating Required Functions

### Import Library

In [49]:
from pathlib import Path
import os
import pandas as pd

### Function to Find the Best YOLO Model (best.pt)

In [50]:
def find_best_model(base_dir='runs_yolo/'):
    best_paths = list(Path(base_dir).rglob('best.pt'))
    if not best_paths:
        raise FileNotFoundError("No 'best.pt' file found in the 'runs/' directory.")
    
    # Optionally, sort by latest modified time
    best_paths.sort(key=lambda p: p.stat().st_mtime, reverse=True)
    
    print(f"[+] Found best.pt at: {best_paths[0]}")
    return str(best_paths[0])


### Functions to Load and Print Metrics from YOLO Training Results

In [51]:
def find_results_csv(directory):
    """Find the results.csv file in the specified directory."""
    for root, dirs, files in os.walk(directory):
        if 'results.csv' in files:
            return os.path.join(root, 'results.csv')
    return None

def load_results_csv(results_csv_path):
    """Load the results CSV into a pandas DataFrame."""
    return pd.read_csv(results_csv_path)

def calculate_total_epochs(df):
    """Calculate the total number of epochs from the DataFrame."""
    return df['epoch'].max()

def calculate_training_loss(epoch_data):
    """Calculate the total training loss from the given epoch data."""
    train_box_loss = epoch_data['train/box_loss']
    train_cls_loss = epoch_data['train/cls_loss']
    train_dfl_loss = epoch_data['train/dfl_loss']
    return train_box_loss + train_cls_loss + train_dfl_loss

def calculate_validation_loss(epoch_data):
    """Calculate the total validation loss from the given epoch data."""
    val_box_loss = epoch_data['val/box_loss']
    val_cls_loss = epoch_data['val/cls_loss']
    val_dfl_loss = epoch_data['val/dfl_loss']
    return val_box_loss + val_cls_loss + val_dfl_loss

def print_final_metrics(df):
    """Print the final metrics for the last epoch."""
    final_epoch_data = df.iloc[-1]

    # Calculate total training and validation loss
    train_loss = calculate_training_loss(final_epoch_data)
    val_loss = calculate_validation_loss(final_epoch_data)

    # Print overall metrics
    print("\n========== Final Training Metrics ==========")
    print(f"Training Loss: {train_loss:.6f}")
    print(f"Precision: {final_epoch_data['metrics/precision(B)']:.6f}")
    print(f"Recall: {final_epoch_data['metrics/recall(B)']:.6f}")
    print(f"mAP@0.5: {final_epoch_data['metrics/mAP50(B)']:.6f}")
    print(f"mAP@0.5:0.95: {final_epoch_data['metrics/mAP50-95(B)']:.6f}")

    print("\n========== Final Validation Metrics ==========")
    print(f"Validation Loss: {val_loss:.6f}")


def print_csv_metrics(directory):
    """Main function to process and print final metrics."""
    # Find the results.csv file
    results_csv_path = find_results_csv(directory)
    
    if not results_csv_path:
        print("Error: 'results.csv' file not found in the specified directory.")
        return

    print(f"Found results.csv at: {results_csv_path}")

    # Load results CSV
    df = load_results_csv(results_csv_path)

    # Get the total number of epochs
    total_epochs = calculate_total_epochs(df)
    print(f"Total number of epochs: {total_epochs}")

    print_final_metrics(df)


### Function to Compare Metrics Between Two YOLO Training Runs

In [52]:
import pandas as pd

def compare_final_metrics(csv1_path, csv2_path):
    df1 = pd.read_csv(csv1_path)
    df2 = pd.read_csv(csv2_path)

    last1 = df1.iloc[-1]
    last2 = df2.iloc[-1]

    metrics_to_compare = {
        "train/box_loss": "Box Loss (Train)",
        "train/cls_loss": "Cls Loss (Train)",
        "train/dfl_loss": "DFL Loss (Train)",
        "metrics/precision(B)": "Precision",
        "metrics/recall(B)": "Recall",
        "metrics/mAP50(B)": "mAP@0.5",
        "metrics/mAP50-95(B)": "mAP@0.5:0.95",
        "val/box_loss": "Box Loss (Val)",
        "val/cls_loss": "Cls Loss (Val)",
        "val/dfl_loss": "DFL Loss (Val)"
    }

    print("Changes in Metrics Before and After Retraning:\n")
    print(f"{'Metric':<25} {'Before':<10} {'After':<10} {'Diff':<10} {'Trend'}")
    print("-" * 65)

    for key, label in metrics_to_compare.items():
        val1 = last1[key]
        val2 = last2[key]
        diff = val2 - val1
        if abs(diff) > 1e-6:
            # If increase, color green; if decrease, color red
            if diff > 0:
                trend = f"\033[92m Increase\033[0m"  # Green
            else:
                trend = f"\033[91m Decrease\033[0m"  # Red

            # Printing with colors
            print(f"{label:<25} {val1:<10.5f} {val2:<10.5f} {diff:<10.5f} {trend}")


### Function to Compare mAP@0.5:0.95 Metrics Between Two Models (Before and After Retraining)

In [66]:
import json

def compare_maps(json_path1, json_path2):
    with open(json_path1, 'r') as f1, open(json_path2, 'r') as f2:
        metrics1 = json.load(f1)
        metrics2 = json.load(f2)

    print("\nmAP@0.5:0.95 Differences Before and After Retraning:\n")
    print(f"{'Class':<15} {'Before':<10} {'After':<10} {'Diff':<10} {'Trend'}")
    print("-" * 60)

    for class_name in metrics1:
        map1 = metrics1[class_name].get("mAP@0.5:0.95", 0)
        map2 = metrics2.get(class_name, {}).get("mAP@0.5:0.95", 0)

        diff = map2 - map1
        if abs(diff) > 1e-6:
            if diff > 0:
                trend = f"\033[92m Increase\033[0m"  # Green for increase
            else:
                trend = f"\033[91m Decrease\033[0m"  # Red for decrease

            # Printing with colors
            print(f"{class_name:<15} {map1:<10.4f} {map2:<10.4f} {diff:<10.4f} {trend}")


### YOLO Model Evaluation and Metrics Extraction

In [54]:
import json
from ultralytics import YOLO

def load_yolo_model(model_path):
    return YOLO(model_path)

def run_model_validation(model, yaml):
    return model.val(data=yaml)

def extract_per_class_metrics(results):
    """
    Extracts mAP@0.5:0.95 per class from results.
    NOTE: Only mAP@0.5:0.95 is available via `results.box.maps`
    """
    per_class_metrics = {}
    if hasattr(results.box, 'maps') and results.box.maps is not None:
        maps = results.box.maps  # This is a NumPy array [num_classes]
        for i, name in results.names.items():
            per_class_metrics[name] = {
                "class_id": i,
                "mAP@0.5:0.95": round(float(maps[i]), 4)
            }
    else:
        print("[-] No per-class mAP@0.5:0.95 data found.")
    return per_class_metrics

def save_metrics_to_json(metrics, output_path):
    with open(output_path, "w") as f:
        json.dump(metrics, f, indent=4)
    print(f"[+] Saved per-class metrics to {output_path}")

def evaluate_and_save_metrics(model_path, yaml, output_json_path="per_class_metrics.json"):
    model = load_yolo_model(model_path)
    results = run_model_validation(model, yaml)
    metrics = extract_per_class_metrics(results)
    save_metrics_to_json(metrics, output_json_path)

### Per-Class mAP@0.5:0.95 Metrics Extraction and Display

In [55]:
import json

def print_per_class_metrics(json_path="per_class_metrics.json"):
    with open(json_path, "r") as f:
        metrics = json.load(f)
    
    print("Per-Class mAP@0.5:0.95 Metrics:\n")
    print(f"{'Class Name':<15} {'Class ID':<10} {'mAP@0.5:0.95':<15}")
    print("-" * 40)
    
    for name, data in metrics.items():
        print(f"{name:<15} {data['class_id']:<10} {data['mAP@0.5:0.95']:<15}")


## Calling Functions

### Before Retraning

In [56]:
yolov8 = './runs/train/yolov8'
print_csv_metrics(yolov8)

Found results.csv at: ./runs/train/yolov8\results.csv
Total number of epochs: 50

========== Final Training Metrics ==========
Training Loss: 2.047820
Precision: 0.643800
Recall: 0.268230
mAP@0.5: 0.282050
mAP@0.5:0.95: 0.177310

========== Final Validation Metrics ==========
Validation Loss: 5.496680


In [57]:
best_pt_path = find_best_model(yolov8)
evaluate_and_save_metrics(best_pt_path, yaml="yolov8.yaml",output_json_path="per_class_metrics.json")

[+] Found best.pt at: runs\train\yolov8\weights\best.pt
Ultralytics 8.3.107  Python-3.12.4 torch-2.5.1+cpu CPU (AMD Ryzen 7 5800H with Radeon Graphics)
Model summary (fused): 72 layers, 3,008,378 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Users\sande\OneDrive\Desktop\IITTP\2_Asgn\Sem2\2_dl\SemProject\Automated_SegmentAndYolo\datasets\new_dataset_yolo_split\val\labels.cache... 1181 images, 1 backgrounds, 0 corrupt: 100%|██████████| 1181/1181 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 74/74 [02:22<00:00,  1.93s/it]


                   all       1181      69465      0.656      0.275       0.34      0.232
                  pool          9          9      0.908          1      0.995      0.851
            vegetation         73       1130      0.318      0.239      0.179      0.107
                  roof         38         59     0.0735      0.576      0.487      0.406
                  wall         52        220      0.166      0.227      0.167      0.116
                window         27         67      0.701      0.179      0.207      0.118
                person         71        523       0.91      0.272      0.419      0.247
                   dog          8         14          1          0      0.186      0.144
                   car       1110      53660       0.84      0.238       0.49      0.245
               bicycle         32         70      0.708      0.485      0.615       0.29
                  tree         31         61      0.915      0.352      0.501      0.368
                 truc

In [58]:
print_per_class_metrics("per_class_metrics.json")

Per-Class mAP@0.5:0.95 Metrics:

Class Name      Class ID   mAP@0.5:0.95   
----------------------------------------
unlabeled       0          0.2322         
pool            1          0.8511         
vegetation      2          0.107          
roof            3          0.4061         
wall            4          0.1156         
window          5          0.1175         
person          6          0.2474         
dog             7          0.1438         
car             8          0.2454         
bicycle         9          0.2903         
tree            10         0.3684         
truck           11         0.0295         
bus             12         0.0            
vehicle         13         0.0959         


### After Retraning

In [59]:
new_path = './runs/train/fine-tune-yolov8'
print_csv_metrics(new_path)

Found results.csv at: ./runs/train/fine-tune-yolov8\results.csv
Total number of epochs: 11

========== Final Training Metrics ==========
Training Loss: 2.637500
Precision: 0.627330
Recall: 0.537950
mAP@0.5: 0.553620
mAP@0.5:0.95: 0.399380

========== Final Validation Metrics ==========
Validation Loss: 2.982540


In [60]:
best_pt_path = find_best_model(new_path)
evaluate_and_save_metrics(best_pt_path, yaml="yolo_retrain.yaml",output_json_path="per_class_metrics_retrain.json")

[+] Found best.pt at: runs\train\fine-tune-yolov8\weights\best.pt
Ultralytics 8.3.107  Python-3.12.4 torch-2.5.1+cpu CPU (AMD Ryzen 7 5800H with Radeon Graphics)


Model summary (fused): 72 layers, 3,008,378 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Users\sande\OneDrive\Desktop\IITTP\2_Asgn\Sem2\2_dl\SemProject\Automated_SegmentAndYolo\datasets\split_videos_dataset\val\labels.cache... 3 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3/3 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.25it/s]


                   all          3          6      0.432      0.583      0.636       0.46
                  wall          1          1          1          0          0          0
                person          1          1      0.137          1      0.995      0.438
                  tree          1          3      0.388      0.333      0.555      0.506
               vehicle          1          1      0.202          1      0.995      0.895
Speed: 1.0ms preprocess, 52.2ms inference, 0.0ms loss, 2.8ms postprocess per image
Results saved to runs\detect\val13
[+] Saved per-class metrics to per_class_metrics_retrain.json


In [61]:
print_per_class_metrics("per_class_metrics_retrain.json")

Per-Class mAP@0.5:0.95 Metrics:

Class Name      Class ID   mAP@0.5:0.95   
----------------------------------------
unlabeled       0          0.4599         
pool            1          0.4599         
vegetation      2          0.4599         
roof            3          0.4599         
wall            4          0.0            
window          5          0.4599         
person          6          0.4378         
dog             7          0.4599         
car             8          0.4599         
bicycle         9          0.4599         
tree            10         0.5064         
truck           11         0.4599         
bus             12         0.4599         
vehicle         13         0.8955         


### Compare both metrics

In [62]:
new_path = './runs/train/fine-tune-yolov8'
old_path = './runs/train/yolov8'

results_csv_path = find_results_csv(new_path)
results_csv_path_1 = find_results_csv(old_path)

compare_final_metrics(results_csv_path_1, results_csv_path)

Changes in Metrics Before and After Retraning:

Metric                    Before     After      Diff       Trend
-----------------------------------------------------------------
Box Loss (Train)          0.73327    0.98916    0.25589     Increase
Cls Loss (Train)          0.45441    0.70747    0.25306     Increase
DFL Loss (Train)          0.86014    0.94087    0.08073     Increase
Precision                 0.64380    0.62733    -0.01647    Decrease
Recall                    0.26823    0.53795    0.26972     Increase
mAP@0.5                   0.28205    0.55362    0.27157     Increase
mAP@0.5:0.95              0.17731    0.39938    0.22207     Increase
Box Loss (Val)            2.02042    0.99684    -1.02358    Decrease
Cls Loss (Val)            2.42410    0.97288    -1.45122    Decrease
DFL Loss (Val)            1.05216    1.01282    -0.03934    Decrease


In [67]:
compare_maps("per_class_metrics.json", "per_class_metrics_retrain.json")


mAP@0.5:0.95 Differences Before and After Retraning:

Class           Before     After      Diff       Trend
------------------------------------------------------------
unlabeled       0.2322     0.4599     0.2277      Increase
pool            0.8511     0.4599     -0.3912     Decrease
vegetation      0.1070     0.4599     0.3529      Increase
roof            0.4061     0.4599     0.0538      Increase
wall            0.1156     0.0000     -0.1156     Decrease
window          0.1175     0.4599     0.3424      Increase
person          0.2474     0.4378     0.1904      Increase
dog             0.1438     0.4599     0.3161      Increase
car             0.2454     0.4599     0.2145      Increase
bicycle         0.2903     0.4599     0.1696      Increase
tree            0.3684     0.5064     0.1380      Increase
truck           0.0295     0.4599     0.4304      Increase
bus             0.0000     0.4599     0.4599      Increase
vehicle         0.0959     0.8955     0.7996      Increase
